### <u> _**Time-Dependent 2D Schrödinger Equation:**_

The two dimensional time dependent Schrödinger equation written in position space for a particle of mass, $m=1$ and $\hbar = 1$ takes the form :

$$
i\frac{\partial {\psi(x,y,t)}}{\partial {t} } = \left[-\frac{1}{2}\nabla^2 + V(x)\right ]\psi(x,y,t)
$$

By definition the Laplacian in cartesian coordinate is , $\nabla^2 = \frac{\partial}{\partial x^2} + \frac{\partial}{\partial y^2}$. 

We discretize our wave function on a square grid such that ($\Delta x = \Delta y $) in the following manner: 


$$
\psi(x,y,t) = \psi(i\Delta x,j\Delta y,l\Delta t) = \psi_{i,j}^{l}  \rightarrow
\begin{bmatrix}
\psi_{1,1}^{l}\\
\psi_{1,2}^{l}\\
\psi_{1,3}^{l}\\
\vdots\\
\psi_{N,N}^{l}
\end{bmatrix}
$$

We recall the expression of the central difference discretization of the Laplacian operator and its associated matrix in 1 dimension:

$$
\frac{\partial}{\partial x^2} \psi(x,y,t) = \frac{\psi_{i+1,j}^{l} - 2\psi_{i,j}^{l} + \psi_{i-1,j}^{l}}{\Delta x^2} \rightarrow d_{xx} = \frac{1}{\Delta x^2}

\begin{bmatrix}
-2 &  1 & &  & 1&  \\
 1  & -2 & 1  & &  \\
   &  1 & -2 & \ddots & \\
   &    &   \ddots & \ddots & 1\\
 1 &    &          & 1 & -2 & \\
\end{bmatrix}
$$

Using the discretization in vector form of our 2D-wavefunction, we have to rewrite the derivative with respect to $x$ in the form of a tensor product:

$$
D_{xx} = d_{xx} \otimes \mathbb{I} = \frac{1}{\Delta x^2}
\begin{bmatrix}
 &d_{xx} & & & & \\
 & &d_{xx} & & & \\
 & & & d_{xx} & & \\
 & & && \ddots& \\
 & & & & &d_{xx} \\\end{bmatrix}
$$

Doing the same procedure for the derivative with respect to $y$ gives us:
$$
D_{yy} = \mathbb{I} \otimes  d_{yy} = \frac{1}{\Delta y^2}
\begin{bmatrix}
-2\mathbb{I} &  \mathbb{I} & &  & \mathbb{I}&  \\
 \mathbb{I}  & -2\mathbb{I} & \mathbb{I}  & &  \\
   &  \mathbb{I} & -2\mathbb{I} & \ddots & \\
   &    &   \ddots & \ddots & \mathbb{I}\\
 \mathbb{I} &    &          & \mathbb{I} & -2\mathbb{I} & \\
\end{bmatrix}
$$

Finally the expression of the Laplacian operator in its discretized form is: 

$$
\Delta = D_{xx} + D_{yy} = d_{xx} \otimes \mathbb{I} + \mathbb{I} \otimes  d_{yy} = d_{xx} + d_{yy} = d_{xx} \oplus d_{yy}
$$

where $\oplus$ is what's known as a Kronecker sum.

The potential matrix evaluated on the grid is rearranged according to our our wave function vector:
$$
V = 
\left[\begin{array}{cccc}
V_{11}&V_{12}&\cdots &V_{1N}\\
V_{21}&V_{22}&\cdots &V_{2N}\\
\vdots & &\ddots &\vdots \\
V_{N1}&V_{N2}&\cdots &V_{NN}\\
\end{array}\right]

\rightarrow
\begin{bmatrix}
 &V_{11} & & & & \\
 & &V_{12} & & & \\
 & & & V_{13} & & \\
 & & & &\ddots & \\
 & & & & &V_{NN} \\
\end{bmatrix}


$$

We can rewrite our final expression for the discretized 2D time dependent Schrödinger equation, by using forward Euler* to time step. 

$$
\psi_{i,j}^{l+1} = \left[ \mathbb{I} +\frac{i \Delta t }{2(\Delta x)^{2}} d_{xx} \oplus d_{yy} 
-i \Delta t V
\right]
\psi_{i,j}^{l}
$$

###### *I actually used RK4 in the code, better stability and error in $O(t^{4})$ (Although still not unitary)

#### <u> _**Free Particle on a square domain: Eigenstates**_

In [ ]:
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.style.use(['classic'])

In [ ]:
def gaussian_wavepacket(x,y,x0 = 25,y0 = 25, sigma =  5,px=10,py=0): # Gaussian Wave Packet of average momentum p0 and width sigma0
    
    phase = np.exp( 1j*(X*px + Y*py))
    px = np.exp( - 1/2 * ((X - x0)**2 + (y - y0)**2)/sigma**2 )
    
    wave_function = phase*px

    return wave_function

def square_well_potential(x,y):
    return x*0


N = 101

x,dx = np.linspace(0,50,N,retstep = True)
y,dy = np.linspace(0,50,N, retstep = True)

dt = 0.1*dx**2

t = np.arange(0,30,dt)

X,Y = np.meshgrid(x,y)

factor = -1/(2*dx**2)

d = np.diag( [-2]*(N),k = 0) + np.diag( [1]*(N-1),k = 1) + np.diag( [1]*(N-1),k = -1) #derivative operator in 1D dimension 


T = scipy.sparse.kronsum(d,d) * factor

V = square_well_potential(X,Y)
V = -scipy.sparse.diags(V.reshape(N**2),(0))

H =  T + V


_,eigenvectors = scipy.sparse.linalg.eigsh(H,k = 10, which = 'SM')


fig,axs = plt.subplots(2,5,figsize = (20,10))



for i in range(5):
    axs[0][i].contourf(abs(eigenvectors.T[i].reshape((N,N)))**2,extent = (0,50,0,50))
    axs[1][i].contourf(abs(eigenvectors.T[i+5].reshape((N,N)))**2,extent = (0,50,0,50))




#### <u> _**Double Slit experiment**_

In [ ]:
T = scipy.sparse.kronsum(d,d) * factor
V = square_well_potential(X,Y)
V[:,60:62]  = 100
V[45:50,60:62] = 0
V[55:60,60:62] = 0

V = -scipy.sparse.diags(V.reshape(N**2),(0))

H =  T + V

psi = np.zeros((len(t),N**2),dtype = 'complex128')
psi0 = gaussian_wavepacket(X,Y,px = -30).reshape(N**2)
psi[0,:] = psi0 




In [ ]:
def f(psi):
    return (scipy.sparse.diags(np.ones(N**2),(0)) + 1j*H).dot(psi)

for l in range(len(t)-1):

    k1 = f(psi[l,:])
    k2 = f(psi[l,:]+dt/2*k1)
    k3 = f(psi[l,:]+dt/2*k2)
    k4 = f(psi[l,:]+dt*k3)

    psi[l+1,:] = psi[l,:] + dt/6 *(k1+2*k2+2*k3+k4)

fig,ax = plt.subplots()

def init():
        im = ax.contourf(X,Y,abs(psi[0,:].real.reshape((N,N)))**2,20,cmap = 'plasma')
        fig.colorbar(im)


def animate(i):
        ax.clear()
        im = ax.contourf(X,Y,abs(psi[i,:].reshape((N,N))),20,cmap = 'plasma')
        fig.suptitle(f'{i*dt:.3}')
        


ani = animation.FuncAnimation(fig, animate, frames =t.size, init_func = init,interval=20,repeat = False)

ani.save('Double_Slit.gif',dpi = 75)

In [ ]:
import matplotlib
matplotlib.rcParams['savefig.dpi']